# Hands-On with LangChain

This notebook demonstrates LangChain's **2025 capabilities** using Azure OpenAI. We'll explore modern LCEL patterns, LangGraph agents (replacing legacy AgentExecutor), Azure Application Insights  integration, and evaluation frameworks.

## 🆕 What's New in 2025:
- **LangGraph** replaces legacy AgentExecutor patterns
- **Azure Application Insights** production-ready monitoring and evaluation  
- **LangChain Sandbox** for safe code execution
- **Modern evaluation frameworks** integration

## Environment Setup

**What this does:** Securely loads API keys and configuration from a `.env` file and validates all required settings for the 2025 LangChain stack.

The code below creates a comprehensive configuration manager that:
- **Loads environment variables** using `python-dotenv` (industry standard for secure credential management)
- **Validates Azure OpenAI credentials** (API key, endpoint, deployment name)
- **Configures Azure Application Insights + OpenTelemetry for monitoring** for production observability
- **Sets up AI Search** for web search capabilities
- **Enables 2025 evaluation frameworks** like Azure ML and MLflow


In [10]:
# Environment and configuration setup with 2025 LangSmith integration
import os
import warnings
from pathlib import Path
from dotenv import load_dotenv, find_dotenv

class NotebookConfig:
    """Azure-centric configuration management for Jupyter notebooks"""

    def __init__(self):
        env_path = find_dotenv()
        if env_path:
            load_dotenv(env_path)
            print(f"✅ Loaded environment from: {env_path}")
        else:
            warnings.warn("No .env file found. Using system environment variables only.")
        
        self._load_azure_config()
        self._load_monitoring_config()
        self._validate_config()

    def _load_azure_config(self):
        """Load Azure service configurations"""
        self.azure_openai_key = os.getenv('AZURE_OPENAI_API_KEY')
        self.azure_openai_endpoint = os.getenv('AZURE_OPENAI_ENDPOINT')
        self.azure_openai_deployment = os.getenv('AZURE_OPENAI_CHAT_DEPLOYMENT_NAME', 'gpt-4o-mini')
        self.azure_openai_version = os.getenv('AZURE_OPENAI_API_VERSION', '2024-05-01-preview')

        self.azure_ai_search_key = os.getenv('AZURE_AI_SEARCH_KEY')
        self.azure_ai_search_endpoint = os.getenv('AZURE_AI_SEARCH_ENDPOINT')
        self.azure_ai_search_index = os.getenv('AZURE_AI_SEARCH_INDEX')

        self.azure_ml_workspace = os.getenv('AZURE_ML_WORKSPACE')
        self.azure_ml_subscription_id = os.getenv('AZURE_ML_SUBSCRIPTION_ID')
        self.azure_ml_resource_group = os.getenv('AZURE_ML_RESOURCE_GROUP')

    def _load_monitoring_config(self):
        """Load Azure Application Insights configuration"""
        self.app_insights_connection_string = os.getenv('APPINSIGHTS_CONNECTION_STRING')
        self.environment = os.getenv('ENVIRONMENT', 'development')
        self.debug = os.getenv('DEBUG', 'false').lower() == 'true'

    def _validate_config(self):
        """Validate required Azure configurations"""
        errors = []

        if not self.azure_openai_key:
            errors.append("AZURE_OPENAI_API_KEY is required")
        if not self.azure_openai_endpoint:
            errors.append("AZURE_OPENAI_ENDPOINT is required")
        if not self.azure_ai_search_key or not self.azure_ai_search_endpoint:
            errors.append("Azure AI Search configuration is incomplete")
        if not self.azure_ml_workspace or not self.azure_ml_subscription_id or not self.azure_ml_resource_group:
            errors.append("Azure ML configuration is incomplete")

        if errors:
            raise ValueError(f"Configuration errors: {', '.join(errors)}")

        print("✅ All required Azure configurations validated successfully")

    def display_config(self):
        """Display current configuration (hiding secrets)"""
        print("Azure OpenAI:")
        print(f"  Endpoint: {self.azure_openai_endpoint}")
        print(f"  Deployment: {self.azure_openai_deployment}")
        print(f"  API Version: {self.azure_openai_version}")
        print(f"  API Key: {'*' * 20 + self.azure_openai_key[-4:] if self.azure_openai_key else 'Not set'}")

        print("\nAzure AI Search:")
        print(f"  Endpoint: {self.azure_ai_search_endpoint}")
        print(f"  Index: {self.azure_ai_search_index}")
        print(f"  API Key: {'*' * 20 + self.azure_ai_search_key[-4:] if self.azure_ai_search_key else 'Not set'}")

        print("\nAzure ML:")
        print(f"  Workspace: {self.azure_ml_workspace}")
        print(f"  Subscription ID: {self.azure_ml_subscription_id}")
        print(f"  Resource Group: {self.azure_ml_resource_group}")

        print("\nMonitoring:")
        print(f"  App Insights Connection String: {'*' * 20 + self.app_insights_connection_string[-4:] if self.app_insights_connection_string else 'Not set'}")
        print(f"  Environment: {self.environment}")
        print(f"  Debug Mode: {self.debug}")

# Initialize configuration
try:
    config = NotebookConfig()
    config.display_config()
except Exception as e:
    print(f"❌ Failed to load configuration: {e}")
    print("\nPlease ensure your .env file includes all required Azure variables.")
    raise

✅ Loaded environment from: c:\Projects\Siemens-Energy\langchain-azure-centric-main - wip\notebooks\.env
✅ All required Azure configurations validated successfully
Azure OpenAI:
  Endpoint: https://oai-pcm-001.openai.azure.com
  Deployment: chat-deployment
  API Version: 2023-05-15
  API Key: ********************iIRs

Azure AI Search:
  Endpoint: https://ais-pcm-001.search.windows.net
  Index: documents-index
  API Key: ********************mGL0

Azure ML:
  Workspace: ml-workspace-pcm
  Subscription ID: "b0e6535c-d468-4bf0-81c2-4812406b7754"  # already present
  Resource Group: rg-sie-energy-pcm-workshop-001

Monitoring:
  App Insights Connection String: ********************cee6
  Environment: development
  Debug Mode: False


### 1.1 Basic Prompting and Model I/O with LCEL

**What this does:** Demonstrates LangChain Expression Language (LCEL) for connecting prompts, models, and output parsers in a composable chain.

The code below creates a basic chain that:
- **Initializes Azure OpenAI chat model** using the configuration from above
- **Creates a prompt template** with variables for dynamic content
- **Uses LCEL pipe operator (`|`)** to compose prompt → model → output parser
- **Parses model response** into clean string output

**Key 2025 Pattern:** LCEL is LangChain's modern composition syntax that replaced legacy chains. It's similar to Unix pipes but for AI workflows.

**LCEL OpenAI Chat:** This pattern works identically with Azure OpenAI - the model provider is abstracted away by LangChain's interface.

In [11]:
%pip install --quiet langchain_openai
%pip install --quiet pydantic pydantic-core


from langchain_openai import AzureChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# Initialize Azure OpenAI with configuration
llm = AzureChatOpenAI(
    azure_deployment=config.azure_openai_deployment,
    api_version=config.azure_openai_version,
    temperature=0.7,
    azure_endpoint=config.azure_openai_endpoint,
    api_key=config.azure_openai_key
) 

prompt = ChatPromptTemplate.from_template("Question: {question}\nAnswer: Let's think step by step.")
chain = prompt | llm | StrOutputParser()
response = chain.invoke({"question": "What is the capital of France?"})
print(response)

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Sure! Let's break it down step by step:

1. France is a country located in Western Europe.
2. Every country has a main city known as its "capital," where the government is usually based.
3. The capital of France is well-known for its history, culture, and landmarks like the Eiffel Tower.
4. The name of this city is Paris.

**Final Answer:** The capital of France is Paris.
Sure! Let's break it down step by step:

1. France is a country located in Western Europe.
2. Every country has a main city known as its "capital," where the government is usually based.
3. The capital of France is well-known for its history, culture, and landmarks like the Eiffel Tower.
4. The name of this city is Paris.

**Final Answer:** The capital of France is Paris.


### 1.2 Sequential Processing with LCEL (Modern Pattern)

**What this does:** Creates a complex multi-step workflow where the output of one LLM call becomes input to the next, using modern LCEL composition patterns.

The code below demonstrates:
- **Multi-step chain creation** with `RunnableParallel` for parallel execution
- **Data passing between steps** using `RunnablePassthrough` to maintain input context
- **Sequential workflow orchestration** where step 1 generates a company name, step 2 creates a catchphrase
- **Modern 2025 pattern replacement** of legacy `SimpleSequentialChain` with LCEL

**Key Innovation:** `RunnableParallel` allows multiple outputs in a single invocation, enabling complex multi-agent-like behaviors.

**Workflow pattern Azure compliant:** This workflow pattern works identically with Azure OpenAI, and could be enhanced with Azure Application Insights for step-by-step monitoring.

In [12]:
from langchain_openai import AzureChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableParallel

# Initialize Azure OpenAI with configuration
llm = AzureChatOpenAI(
    azure_deployment=config.azure_openai_deployment,
    api_version=config.azure_openai_version,
    temperature=0.7,
    azure_endpoint=config.azure_openai_endpoint,
    api_key=config.azure_openai_key
) 

# Modern LCEL approach - compose operations with pipe operator
name_prompt = ChatPromptTemplate.from_template(
    "What is a good name for a company that makes {product}?"
)

catchphrase_prompt = ChatPromptTemplate.from_template(
    "Write a creative catchphrase for the following company: {company_name}"
)

# Build sequential chain using LCEL composition
name_chain = name_prompt | llm | StrOutputParser()

# Create a more complex chain that passes results between steps
def create_sequential_chain():
    """Creates a sequential chain using modern LCEL patterns"""
    
    # Step 1: Generate company name
    step1 = (
        {"product": RunnablePassthrough()} 
        | name_prompt 
        | llm 
        | StrOutputParser()
    )
    
    # Step 2: Generate catchphrase using the company name
    step2 = (
        {"company_name": step1}
        | catchphrase_prompt
        | llm
        | StrOutputParser()
    )
    
    # Combine results into final output
    return RunnableParallel({
        "company_name": step1,
        "catchphrase": step2,
        "product": RunnablePassthrough()
    })

# Execute the sequential chain
sequential_chain = create_sequential_chain()
product = "colorful, eco-friendly socks"

print(f"Input: {product}")
print("\n🔗 Running sequential LCEL chain...")

result = sequential_chain.invoke(product)
print(f"\n✅ Results:")
print(f"Product: {result['product']}")
print(f"Company Name: {result['company_name']}")
print(f"Catchphrase: {result['catchphrase']}")

Input: colorful, eco-friendly socks

🔗 Running sequential LCEL chain...

✅ Results:
Product: colorful, eco-friendly socks
Company Name: Absolutely! Here are some name ideas for a company that makes colorful, eco-friendly socks:

1. **EcoSole Socks**
2. **VividStep**
3. **GreenToes**
4. **Colorleaf Socks**
5. **SockSprout**
6. **EcoChroma**
7. **BrightRoots Socks**
8. **SoleBloom**
9. **EarthHue Socks**
10. **HappyEarth Socks**
11. **RainbowRoots**
12. **EcoJive Socks**
13. **Sprout & Stitch**
14. **LeafyLoops**
15. **PureHue Socks**

Let me know if you’d like names in a particular style or with certain words included!
Catchphrase: Absolutely! Here are some creative catchphrases for a colorful, eco-friendly sock company (using your name ideas):

---

**1. EcoSole Socks**  
*“Step Bright. Tread Light.”*

**2. GreenStep Socks**  
*“Every Step, a Greener Tomorrow.”*

**3. ChromaSole**  
*“Color Your World, Conserve Your Earth.”*

**4. VividLeaf Socks**  
*“Vivid Color. Verdant Conscience.”

### 1.3 Streaming with LCEL

**What this does:** Implements real-time token streaming for improved user experience, displaying AI responses as they're generated rather than waiting for completion.

The code below creates:
- **Custom callback handler** that intercepts each token as it's generated by the LLM
- **Streaming chain configuration** using `.with_config()` to attach the callback
- **Real-time token display** printing each word/token immediately upon generation
- **Production-ready streaming pattern** for chat interfaces and interactive applications

**Key 2025 Feature:** Native streaming support is built into LCEL, making it effortless to add real-time responses.

**Azure OpenAI streaming support:** Azure OpenAI supports streaming natively. For enterprise monitoring, combine with Azure Application Insights to track streaming performance and token usage in real-time.

In [13]:
# Required imports
import os
from langchain_openai import AzureChatOpenAI
from langchain_core.callbacks import BaseCallbackHandler
from langchain_core.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser


# Define a custom streaming callback handler
class StreamingCallbackHandler(BaseCallbackHandler):
    """Custom callback handler to demonstrate streaming"""
    def on_llm_new_token(self, token: str, **kwargs) -> None:
        print(token, end="", flush=True)

# Initialize the AzureChatOpenAI model with streaming enabled
llm = AzureChatOpenAI(
    azure_deployment=config.azure_openai_deployment,
    api_version=config.azure_openai_version,
    temperature=0.7,
    azure_endpoint=config.azure_openai_endpoint,
    api_key=config.azure_openai_key,
    streaming=True,
    callbacks=[StreamingCallbackHandler()]
)

# Create a prompt template
streaming_prompt = ChatPromptTemplate.from_template(
    "Write a brief story about {topic}. Make it engaging and creative."
)

# Build the streaming chain
streaming_chain = (
    streaming_prompt
    | llm
    | StrOutputParser()
)

# Invoke the chain
print("🌊 Streaming response for a story about 'space exploration':")
print("=" * 50)

response = streaming_chain.invoke({"topic": "space exploration"})

print("\n" + "=" * 50)
print("✅ Streaming complete!")

🌊 Streaming response for a story about 'space exploration':
Captain Mira Chen’s boots echoed through the humming corridorCaptain Mira Chen’s boots echoed through the humming corridor of of the starship *Odyssey*. Beyond the the starship *Odyssey*. Beyond the reinforced windows reinforced windows, Saturn’s rings glimmered like cosmic, Saturn’s rings glimmered like cosmic jewelry, silent and jewelry, silent and eternal.

Her mission was simple eternal.

Her mission was simple, yet daunting: chart a route through, yet daunting: chart a route through the Perseid Nebula, an uncharted swirl the Perseid Nebula, an uncharted swirl of stardust rumored to of stardust rumored to hide new worlds. The nebula’s colors hide new worlds. The nebula’s colors danced danced across the hull—violet, emerald, across the hull—violet, emerald, and and gold—casting prismatic shadows.

 gold—casting prismatic shadows.

As theAs the ship plunged into the nebula ship plunged into the nebula, the sensors, the senso

### 1.4 Function Calling and Basic Agents

**What this does:** Creates an intelligent agent using 2025's LangGraph framework (replacing legacy AgentExecutor) that can use tools and maintain conversation memory.

The code below demonstrates:
- **Modern 2025 LangGraph agent** using `create_react_agent` (replaces deprecated AgentExecutor)
- **Tool integration** with Azure AI Search for enterprise search capabilities, calculator, and text formatting functions
- **Memory persistence** using `MemorySaver` for conversation threading
- **Streaming execution** with real-time step visibility for debugging
- **Multi-step reasoning** where the agent plans, uses tools, and synthesizes results

**Key 2025 Migration:** `create_react_agent` from LangGraph is the modern replacement for the legacy AgentExecutor pattern.

**Considerations:** 
- Use **Azure Cosmos DB** instead of memory for production-grade conversation persistence
- Monitor with **Azure Application Insights** for enterprise observability

In [14]:
# Install required packages
%pip install --quiet langgraph azure-search-documents opencensus-ext-azure

from langchain.tools import tool
from langgraph.prebuilt import create_react_agent
from langgraph.checkpoint.memory import MemorySaver
from langchain_openai import AzureChatOpenAI
from azure.search.documents import SearchClient
from azure.core.credentials import AzureKeyCredential
from opencensus.ext.azure.log_exporter import AzureLogHandler
import logging

# Initialize Azure OpenAI
llm_agent = AzureChatOpenAI(
    azure_deployment=config.azure_openai_deployment,
    api_version=config.azure_openai_version,
    temperature=0,
    azure_endpoint=config.azure_openai_endpoint,
    api_key=config.azure_openai_key
)

# Setup Azure Application Insights logging
logger = logging.getLogger(__name__)
logger.addHandler(AzureLogHandler(connection_string=os.getenv("APPINSIGHTS_CONNECTION_STRING")))
logger.setLevel(logging.INFO)
logger.info("✅ Azure Application Insights logging initialized.")

# Define Azure AI Search tool
@tool
def azure_ai_search(query: str) -> str:
    """Search enterprise content using Azure AI Search."""
    try:
        search_client = SearchClient(
            endpoint=config.azure_ai_search_endpoint,
            index_name=config.azure_ai_search_index,
            credential=AzureKeyCredential(config.azure_ai_search_key)
        )
        results = search_client.search(query, top=2)
        output = "\n".join([doc["content"] for doc in results])
        logger.info(f"Azure AI Search query: {query}")
        logger.info(f"Azure AI Search results: {output}")
        return output or "No results found."
    except Exception as e:
        logger.error(f"Azure AI Search error: {e}")
        return f"Search failed: {e}"

# Define other tools
@tool
def calculate(expression: str) -> str:
    """Evaluate basic math expressions."""
    try:
        allowed_chars = set('0123456789+-*/().** ')
        if not all(c in allowed_chars for c in expression):
            return "Error: Only basic mathematical operations are allowed"
        logger.info(f"Azure AI Search query: {expression}")
        logger.info(f"Azure AI Search results: {allowed_chars}")
        return f"Result: {eval(expression)}"
    except Exception as e:
        logger.error(f"Azure AI Search error: {e}")
        return f"Error evaluating expression: {e}"

@tool 
def format_text(text: str, style: str = "upper") -> str:
    """Format text in upper, lower, or title case."""
    return {
        "upper": text.upper(),
        "lower": text.lower(),
        "title": text.title()
    }.get(style, f"Unknown style: {style}")

# Combine tools
tools = [azure_ai_search, calculate, format_text]

# LangGraph agent setup
system_message = "You are a helpful assistant that can search enterprise content, calculate, and format text. Explain your reasoning step by step."
memory = MemorySaver()

agent = create_react_agent(
    model=llm_agent,
    tools=tools,
    prompt=system_message,
    checkpointer=memory
)

# Run a test query
print("🤖 Testing Azure-centric LangGraph agent...")
config = {"configurable": {"thread_id": "demo-conversation"}}
query = "Calculate 16 raised to the power of 0.5, then format the result as 'The answer is X' in title case"

for step in agent.stream(
    {"messages": [("user", query)]},
    config=config,
    stream_mode="updates"
):
    if step:
        print(f"📝 Step: {step}")

print("\n" + "=" * 50)
print("✅ MODERN 2025 FEATURES DEMONSTRATED:")
print("   🔄 LangGraph agent (replaces legacy AgentExecutor)")
print("   💾 Built-in memory with conversation threading")
print("   🌊 Real-time streaming execution")
print("   🔍 Azure Application Insights tracing")
print("="* 50)

Note: you may need to restart the kernel to use updated packages.
🤖 Testing Azure-centric LangGraph agent...
📝 Step: {'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_Q4w4cbBOEUSqyWLmEQDJMgHg', 'function': {'arguments': '{"expression": "16^0.5"}', 'name': 'calculate'}, 'type': 'function'}, {'id': 'call_XuGrtqtn5yn7sH2VhYype9r4', 'function': {'arguments': '{"text": "The answer is X", "style": "title"}', 'name': 'format_text'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 55, 'prompt_tokens': 151, 'total_tokens': 206, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-2025-04-14', 'system_fingerprint': 'fp_9ab7d013ff', 'id': 'chatcmpl-CIpki56dUNubb4PqaUMhFimddfazp', 'service_tier': None, 'finish_reason': 'tool_calls', 'l

## 🎯 2025 Evaluation & Monitoring Integration

**What this does:** Integrates production-grade evaluation and monitoring using the leading 2025 frameworks to ensure AI application quality and performance.

The code below demonstrates:
- **Azure Machine Learning + Prompt Flow integration** - for LLM evaluation and metrics
- **Azure Application Insights monitoring** - production tracing and evaluation for LangChain applications
- **Automated evaluation metrics** including Answer Relevancy, Faithfulness, and custom scoring
- **Cost tracking and token usage** monitoring for production optimization
- **Unit test-style evaluation** using pytest-compatible DeepEval patterns


In [15]:
# 2025 PATTERN: Modern evaluation framework integration
%pip install --quiet azure.ai.ml 

import os
import logging
from opencensus.ext.azure.log_exporter import AzureLogHandler
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential

# Setup Azure Application Insights logging
app_insights_conn = os.getenv("APPINSIGHTS_CONNECTION_STRING")
if app_insights_conn:
    logger = logging.getLogger("azure_monitoring")
    logger.addHandler(AzureLogHandler(connection_string=app_insights_conn))
    logger.setLevel(logging.INFO)
    logger.info("✅ Azure Application Insights logging enabled")
else:
    print("⚠️ Application Insights not configured (add APPINSIGHTS_CONNECTION_STRING to enable)")

# Setup Azure ML client for Prompt Flow and experiment tracking
try:
    ml_client = MLClient(
        credential=DefaultAzureCredential(),
        subscription_id=os.getenv("AZURE_ML_SUBSCRIPTION_ID"),
        resource_group=os.getenv("AZURE_ML_RESOURCE_GROUP"),
        workspace_name=os.getenv("AZURE_ML_WORKSPACE")
    )
    print("✅ Azure ML client initialized")
    print(f"📊 Workspace: {ml_client.workspace_name}")
except Exception as e:
    print(f"❌ Failed to initialize Azure ML client: {e}")
    print("Please ensure AZURE_ML_SUBSCRIPTION_ID, AZURE_ML_RESOURCE_GROUP, and AZURE_ML_WORKSPACE are set.")

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
azureml-mlflow 1.60.0 requires azure-storage-blob<=12.19.0,>=12.5.0, but you have azure-storage-blob 12.26.0 which is incompatible.
Overriding of current TracerProvider is not allowed
Overriding of current LoggerProvider is not allowed
Overriding of current LoggerProvider is not allowed
Overriding of current MeterProvider is not allowed
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented
Overriding of current MeterProvider is not allowed
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented
Attempting to instr

Note: you may need to restart the kernel to use updated packages.
✅ Azure ML client initialized
📊 Workspace: ml-workspace-pcm


In [16]:
# 2025 PATTERN: Azure ML integration for comprehensive testing
%pip install --quiet mlflow azureml-mlflow azureml azureml-core
import os
import mlflow
from azure.identity import AzureCliCredential
from azure.ai.ml import MLClient

# Authenticate and connect to Azure ML workspace
ml_client = MLClient(
    credential=AzureCliCredential(),
    subscription_id=os.getenv("AZURE_ML_SUBSCRIPTION_ID"),
    resource_group=os.getenv("AZURE_ML_RESOURCE_GROUP"),
    workspace_name=os.getenv("AZURE_ML_WORKSPACE")
)

# Set MLflow tracking URI manually
tracking_uri = os.getenv("AZURE_MLFLOW_TRACKING_URI") 
mlflow.set_tracking_uri(tracking_uri)
mlflow.set_experiment("LangGraph-Evaluation")

# Simulated evaluation logic
input_text = "What are the main benefits of LangGraph?"
actual_output = "LangGraph provides state management, complex agent workflows, and streaming capabilities for multi-agent systems."
expected_output = "LangGraph offers state management and multi-agent orchestration capabilities."
context = ["LangGraph is LangChain's framework for building stateful, multi-agent applications"]

# Start MLflow run
with mlflow.start_run():
    mlflow.log_param("input", input_text)
    mlflow.log_param("expected_output", expected_output)
    mlflow.log_param("actual_output", actual_output)
    mlflow.log_param("context", str(context))

    # Simulated scoring logic
    relevancy_score = 0.85
    faithfulness_score = 0.9

    mlflow.log_metric("relevancy", relevancy_score)
    mlflow.log_metric("faithfulness", faithfulness_score)

    print("✅ Evaluation logged to Azure ML via MLflow")

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
azure-storage-file-datalake 12.21.0 requires azure-storage-blob>=12.26.0, but you have azure-storage-blob 12.19.0 which is incompatible.
Overriding of current TracerProvider is not allowed
Overriding of current LoggerProvider is not allowed
Overriding of current LoggerProvider is not allowed
Overriding of current MeterProvider is not allowed
Attempting to instrument while already instrumented
Overriding of current MeterProvider is not allowed
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented
Attempting to 

Note: you may need to restart the kernel to use updated packages.
✅ Evaluation logged to Azure ML via MLflow
✅ Evaluation logged to Azure ML via MLflow
🏃 View run happy_flower_2hhwpwd2 at: https://swedencentral.api.azureml.ms/mlflow/v2.0/subscriptions/b0e6535c-d468-4bf0-81c2-4812406b7754/resourceGroups/rg-sie-energy-pcm-workshop-001/providers/Microsoft.MachineLearningServices/workspaces/ml-workspace-pcm/#/experiments/94eae68a-0fbf-4b93-9d5c-2f23ecd93429/runs/4700746e-3a3b-4f37-a72e-3bb5fa190878
🧪 View experiment at: https://swedencentral.api.azureml.ms/mlflow/v2.0/subscriptions/b0e6535c-d468-4bf0-81c2-4812406b7754/resourceGroups/rg-sie-energy-pcm-workshop-001/providers/Microsoft.MachineLearningServices/workspaces/ml-workspace-pcm/#/experiments/94eae68a-0fbf-4b93-9d5c-2f23ecd93429
🏃 View run happy_flower_2hhwpwd2 at: https://swedencentral.api.azureml.ms/mlflow/v2.0/subscriptions/b0e6535c-d468-4bf0-81c2-4812406b7754/resourceGroups/rg-sie-energy-pcm-workshop-001/providers/Microsoft.Machin

In [17]:
# 2025 PATTERN: Cost tracking and token usage monitoring
%pip install --quiet langchain_community

import os
import logging
from datetime import datetime
from opencensus.ext.azure.log_exporter import AzureLogHandler
from langchain.callbacks import get_openai_callback


# Setup Azure Application Insights logging
logger = logging.getLogger("azure_agent_cost_logger")
logger.addHandler(AzureLogHandler(connection_string=os.getenv("APPINSIGHTS_CONNECTION_STRING")))
logger.setLevel(logging.INFO)


def run_agent_with_cost_tracking(agent, query):
    """Run agent with comprehensive cost and performance tracking"""
    
    with get_openai_callback() as cb:
        # Configure conversation with thread ID for tracking
        config = {"configurable": {"thread_id": "cost-tracking-demo"}}
        
        # Run the agent with streaming
        response = None
        for chunk in agent.stream({"messages": [("human", query)]}, config):
            if "agent" in chunk:
                response = chunk["agent"]["messages"][-1].content


     # Log to Application Insights
    logger.info("Agent run completed", extra={
        "custom_dimensions": {
            "timestamp": datetime.utcnow().isoformat(),
            "query": query,
            "response": response,
            "total_tokens": cb.total_tokens,
            "prompt_tokens": cb.prompt_tokens,
            "completion_tokens": cb.completion_tokens,
            "estimated_cost_usd": cb.total_cost
        }
    })
   
    # Display cost metrics
    print("\n💰 Cost Analysis:")
    print(f"  Total Tokens: {cb.total_tokens}")
    print(f"  Prompt Tokens: {cb.prompt_tokens}")
    print(f"  Completion Tokens: {cb.completion_tokens}")
    print(f"  Total Cost: ${cb.total_cost:.4f}")
    
    return response, {
        'total_tokens': cb.total_tokens,
        'cost': cb.total_cost,
        'prompt_tokens': cb.prompt_tokens,
        'completion_tokens': cb.completion_tokens
    }

# Example usage with cost tracking
if 'agent' in locals():
    #test_query = "What's the current weather in San Francisco?"
    test_query = "What's the general climate information of New York during May?"
    response, metrics = run_agent_with_cost_tracking(agent, test_query)
    
    print(f"\n🤖 Agent Response: {response}")
    print(f"📊 Performance Metrics: {metrics}")
else:
    print("⚠️ Agent not initialized - run the previous cells first")

Note: you may need to restart the kernel to use updated packages.

💰 Cost Analysis:
  Total Tokens: 334
  Prompt Tokens: 135
  Completion Tokens: 199
  Total Cost: $0.0019

🤖 Agent Response: In May, New York (specifically New York City) typically experiences mild to warm spring weather. Here’s a general overview of the climate during this month:

- **Average High Temperature:** 68°F (20°C)
- **Average Low Temperature:** 53°F (12°C)
- **Rainfall:** May is moderately wet, with an average of about 4 inches (100 mm) of rain spread over 11-12 days.
- **Humidity:** Humidity levels are comfortable, not as high as in the summer.
- **Daylight:** Days are longer, with sunrise around 5:30-6:00 AM and sunset around 8:00 PM by the end of the month.
- **Weather:** The weather is generally pleasant, with a mix of sunny and cloudy days. Occasional rain showers are common, but prolonged rain is rare.

May is considered one of the best months to visit New York due to the comfortable temperatures and blo

## 🎯 Azure Application Insights for Kusto Query

**Kusto Queries:** - access the Azure Application Insights Logs and run kusto queries to retrieve metrics 
- open Azure Application Insights
- on the left, click on Logs.
- click on New Query
- select KQL Mode

**Run the kusto query**:
-     traces
-        | where customDimensions contains "estimated_cost_usd"
-        | project timestamp, customDimensions.query, customDimensions.total_tokens, customDimensions.estimated_cost_usd
-        | order by timestamp desc

**Result**:
- timestamp [UTC] = 2025-09-16T23:56:09.550352Z
- customDimensions_query = What's the general climate information of San Francisco during May?
- customDimensions_total_tokens = 502
- customDimensions_estimated_cost_usd = 0.0022040000000000002




**Reference: Kusto Query Language overview**:
- https://learn.microsoft.com/en-us/kusto/query/?view=microsoft-fabric

---------------------------------------------------------------------------------------------------------------------------------


## 🎯 Reference:



**Microsoft Azure Monitor Opentelemetry Exporter Trace Python Samples**
- https://learn.microsoft.com/en-us/samples/azure/azure-sdk-for-python/microsoft-azure-monitor-opentelemetry-exporter-trace-python-samples/

**Monitor OpenAI Agents SDK with Application Insights:**:
- https://techcommunity.microsoft.com/blog/azure-ai-foundry-blog/monitor-openai-agents-sdk-with-application-insights/4393949

**Monitor Azure OpenAI**:
- https://learn.microsoft.com/en-us/azure/ai-foundry/openai/how-to/monitor-openai

**Integrating Azure Application Insights using opentelemetry-instrumentation in the Sample-app-aoai-chatGPT**:
- https://github.com/microsoft/sample-app-aoai-chatGPT/issues/495

**Visualize traces on Azure AI Foundry Tracing UI**:
- https://learn.microsoft.com/en-us/semantic-kernel/concepts/enterprise-readiness/observability/telemetry-with-azure-ai-foundry-tracing

---------------------------------------------------------------------------------------------------------------------------------

## Summary: LangChain Fundamentals (2025 Edition)

This notebook covered the core concepts of LangChain using modern patterns:

### Key Concepts Learned:
1. **LCEL (LangChain Expression Language)**: Modern composition using pipe operators
2. **Sequential Processing**: Chaining operations with result passing between steps
3. **Streaming**: Real-time token streaming for better user experience  
4. **LangGraph Agents**: Modern agent framework replacing legacy AgentExecutor
5. **Evaluation Integration**: Azure Apllication Insights monitoring and Azure ML + Prompt Flow for testing frameworks
6. **Cost Tracking**: Production-ready token usage and cost monitoring

### 2025 LangChain Evolution:
- **LangGraph**: State management and multi-agent orchestration
- **Modern Agent Patterns**: Memory-enabled agents with conversation threading
- **Evaluation Frameworks**: Comprehensive testing with Azure ML integration
- **Production Monitoring**: Built-in cost tracking and performance metrics

### 2025 Production Considerations:
- Enable Azure Application Insights for tracing for production monitoring
- Implement Azure ML for comprehensive agent testing
- Use conversation threading for memory-enabled applications
- Monitor costs with built-in callback handlers
- Leverage LangGraph for complex multi-agent orchestration

## 🎯 Appendix 1
--------------------------------------------------------------------------------------------------------------------
An example to show an application using Opentelemetry tracing api and sdk. 
Custom dependencies are:
- tracked via spans 
- telemetry is exported to application insights with the AzureMonitorTraceExporter


In [18]:
# Copyright (c) Microsoft Corporation. All rights reserved.
# Licensed under the MIT License.

%pip install --quiet openinference-instrumentation-langchain opentelemetry-api opentelemetry-sdk opentelemetry-exporter-otlp-proto-http

# mypy: disable-error-code="attr-defined"
import os
from opentelemetry import trace
from opentelemetry.sdk.trace import TracerProvider
from opentelemetry.sdk.trace.export import BatchSpanProcessor
from azure.monitor.opentelemetry.exporter import AzureMonitorTraceExporter
from openinference.instrumentation.langchain import LangChainInstrumentor
from opentelemetry import trace as trace_api
from opentelemetry.exporter.otlp.proto.http.trace_exporter import OTLPSpanExporter
from opentelemetry.sdk import trace as trace_sdk
from opentelemetry.sdk.trace.export import ConsoleSpanExporter, SimpleSpanProcessor
from dotenv import load_dotenv
load_dotenv()#'azure.env')

exporter = AzureMonitorTraceExporter.from_connection_string(
    os.getenv("APPINSIGHTS_CONNECTION_STRING")
)

tracer_provider = TracerProvider()

trace_api.set_tracer_provider(tracer_provider)
trace.set_tracer_provider(tracer_provider)
tracer = trace.get_tracer(__name__)
span_processor = BatchSpanProcessor(exporter, schedule_delay_millis=60000)
trace.get_tracer_provider().add_span_processor(span_processor)
LangChainInstrumentor().instrument()

from langchain_openai import AzureChatOpenAI
from langchain.chains import LLMChain
from langchain_core.prompts import PromptTemplate

prompt_template = "Tell me a {adjective} joke"
prompt = PromptTemplate(input_variables=["adjective"], template=prompt_template)
llm = AzureChatOpenAI(api_key = os.getenv('AZURE_OPENAI_API_KEY'),
                      azure_endpoint = os.getenv('AZURE_OPENAI_ENDPOINT'), 
                      api_version = '2024-06-01', 
                      model= os.getenv('AZURE_OPENAI_DEPLOYMENT_NAME'))

chain = LLMChain(llm=llm, prompt=prompt, metadata={"category": "jokes"})
completion = chain.predict(adjective="funny", metadata={"variant": "funny"})
print(completion)

Overriding of current TracerProvider is not allowed
Overriding of current TracerProvider is not allowed
Attempting to instrument while already instrumented
Overriding of current TracerProvider is not allowed
Attempting to instrument while already instrumented


Note: you may need to restart the kernel to use updated packages.
Sure! Here you go:

Why did the scarecrow win an award?  
Because he was outstanding in his field!
Sure! Here you go:

Why did the scarecrow win an award?  
Because he was outstanding in his field!


--------------------------------------------------------------------------------------------------------------------

## 🎯 Appendix 2
--------------------------------------------------------------------------------------------------------------------
**Azure API MAnagement / MCP Servers / Policies**

- Overview of AI gateway capabilities in Azure API Management
https://learn.microsoft.com/en-us/azure/api-management/genai-gateway-capabilities

- About MCP servers in Azure API Management
https://learn.microsoft.com/en-us/azure/api-management/mcp-server-overview 
</br></br>
  Limitations: 
</br>
       Expose and govern an existing MCP server
       https://learn.microsoft.com/en-us/azure/api-management/expose-existing-mcp-server#limitations
</br>
       Expose REST API in API Management as an MCP server
</br>
       https://learn.microsoft.com/en-us/azure/api-management/export-rest-mcp-server#limitations

- Secure access to MCP servers in API Management
https://learn.microsoft.com/en-us/azure/api-management/secure-mcp-servers#steps-to-configure-oauth-2-based-outbound-access

- MCP Center
https://mcp.azure.com/

- MCP Inspector
https://modelcontextprotocol.io/docs/tools/inspector

- APIM ❤️ MCP
https://github.com/Azure-Samples/AI-Gateway/tree/main/labs/mcp-client-authorization

- Policies in Azure API Management
https://learn.microsoft.com/en-us/azure/api-management/api-management-howto-policies

- Protect your API
https://learn.microsoft.com/en-us/azure/api-management/transform-api

- Policies to validate header context
https://learn.microsoft.com/en-us/azure/api-management/secure-mcp-servers

- Use named values in Azure API Management policies
https://learn.microsoft.com/en-us/azure/api-management/api-management-howto-properties?tabs=azure-portal

-  Mock API responses
https://learn.microsoft.com/en-us/azure/api-management/mock-api-responses?tabs=azure-portal

- Debug your APIs using request tracing
https://learn.microsoft.com/en-us/azure/api-management/api-management-howto-api-inspector?WT.mc_id=Portal-Microsoft_Azure_ApiManagement#enable-tracing-for-an-api


--------------------------------------------------------------------------------------------------------------------